# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
from scipy.stats import linregress
import seaborn as sns

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
os.getcwd()

'C:\\Users\\Bauyrjan.Jyenis'

In [3]:
path="C:\\Users\\Bauyrjan.Jyenis\\GuidedCapstone\\data"
os.chdir(path)

In [4]:
os.getcwd()

'C:\\Users\\Bauyrjan.Jyenis\\GuidedCapstone\\data'

In [5]:
os.listdir()

['clean_data.csv',
 'GuidedCapstone',
 'step2_output.csv',
 'step2_output_v1.0.csv',
 'step2_output_v1.1.csv',
 'step2_output_v1.2.csv',
 'step3_output.csv',
 'updated_ski_data.csv']

In [86]:
df = pd.read_csv('step3_output.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,0,2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.00000,152.0,30.0,0
1,1,4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.00000,104.0,80.0,1
2,2,5,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,...,88.0,100.0,110.0,84.0,300.0,47.0,47.00000,107.0,0.0,1
3,3,8,Boreal Mountain Resort,California,7700,500,0,0.0,0,1,...,380.0,200.0,150.0,54.0,400.0,49.0,64.16681,150.0,200.0,1
4,4,9,Dodge Ridge,California,8200,1600,0,0.0,0,0,...,862.0,100.0,114.0,69.0,350.0,78.0,78.00000,140.0,0.0,1


In [87]:
df[df['Name'].str.contains('Big Mountain')]

,Unnamed: 0,Unnamed: 0.1,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
173,173,151,Big Mountain Resort,Montana,6817,2353,0,0.0,0,3,...,3000.0,600.0,123.0,72.0,333.0,81.0,81.0,123.0,600.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [88]:
dfo = pd.DataFrame(df['state'])
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 62 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            174 non-null    int64  
 1   Unnamed: 0.1          174 non-null    int64  
 2   Name                  174 non-null    object 
 3   summit_elev           174 non-null    int64  
 4   vertical_drop         174 non-null    int64  
 5   trams                 174 non-null    int64  
 6   fastEight             174 non-null    float64
 7   fastSixes             174 non-null    int64  
 8   fastQuads             174 non-null    int64  
 9   quad                  174 non-null    int64  
 10  triple                174 non-null    int64  
 11  double                174 non-null    int64  
 12  surface               174 non-null    int64  
 13  total_chairs          174 non-null    int64  
 14  Runs                  174 non-null    float64
 15  TerrainParks          1

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [90]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

#### Let's write a function that standardizes the magnitude of numeric features, we will use this for multiple times

In [91]:
def stdz(x):
    # Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
    scaler = preprocessing.StandardScaler().fit(X)
    # Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X
    X_scaled=scaler.transform(X)
    return X_scaled

In [92]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df['AdultWeekend'] 

In [93]:
X_scaled = stdz(X)

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [94]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

#### Let's write a function that splits the data set into training and testing data sets

In [95]:
def split_data(x, y):
    # Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
    y = y.ravel()
    # Call the train_test_split() function with the first two parameters set to X_scaled and y 
    # Declare four variables, X_train, X_test, y_train and y_test separated by commas 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)
    return X_train, X_test, y_train, y_test

In [96]:
X_train, X_test, y_train, y_test = split_data(X_scaled, y)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [97]:
# all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model_1 = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [98]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model_1.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [99]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

#### Let's write a function that evaluate the prediction model based on explained variance score and mean absolute error, we will use this function multiple times

In [100]:
def evaluate(test, pred):
    evs = explained_variance_score(test, pred)
    print('Explained Variance Score: ', evs)
    print('\n')
    mae = mean_absolute_error(test, pred)
    print('Mean Absolute Error: ', mae)

In [101]:
evaluate(y_test, y_pred)

Explained Variance Score:  -2.7002663303137993e+19


Mean Absolute Error:  22096951519.566845


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [102]:
model_1.intercept_

-3843122103.7112317

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [103]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by=['Coefficient'], ascending=False).head(12)

,Coefficient
trams,7.766754e+11
total_chairs,4.946489e+11
fastSixes,4.936972e+11
double,2.592500e+11
surface,2.530148e+11
triple,2.248076e+11
quad,1.243446e+11
state_New York,1.241076e+11
state_Michigan,1.092112e+11
fastQuads,1.089343e+11


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [104]:
df_new=df.loc[:, ~df.columns.str.contains('state')]

In [105]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         174 non-null    int64  
 1   Unnamed: 0.1       174 non-null    int64  
 2   Name               174 non-null    object 
 3   summit_elev        174 non-null    int64  
 4   vertical_drop      174 non-null    int64  
 5   trams              174 non-null    int64  
 6   fastEight          174 non-null    float64
 7   fastSixes          174 non-null    int64  
 8   fastQuads          174 non-null    int64  
 9   quad               174 non-null    int64  
 10  triple             174 non-null    int64  
 11  double             174 non-null    int64  
 12  surface            174 non-null    int64  
 13  total_chairs       174 non-null    int64  
 14  Runs               174 non-null    float64
 15  TerrainParks       174 non-null    float64
 16  LongestRun_mi      174 non

In [106]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df_new.drop(['Name','AdultWeekend', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df_new['AdultWeekend'] 

In [107]:
# Let's standardize the magnitude by calling stdx() function we have written above
X_scaled = stdz(X)

In [108]:
# Let's split data by calling split_data() function we have written above
X_train, X_test, y_train, y_test = split_data(X_scaled, y)

In [109]:
model_2 = lm.fit(X_train,y_train)

In [110]:
y_pred = model_2.predict(X_test)

In [111]:
evaluate(y_test, y_pred)

Explained Variance Score:  0.7422631715134997


Mean Absolute Error:  6.878831863221978


In [112]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by=['Coefficient'], ascending=False).head(12)

,Coefficient
AdultWeekday,8.973775
summit_elev,3.570798
vertical_drop,2.933940
clusters,2.587249
daysOpenLastYear,1.794499
SkiableTerrain_ac,1.783608
triple,1.706467
LongestRun_mi,1.524131
surface,1.349633
quad,1.292370


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

In [113]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         174 non-null    int64  
 1   Unnamed: 0.1       174 non-null    int64  
 2   Name               174 non-null    object 
 3   summit_elev        174 non-null    int64  
 4   vertical_drop      174 non-null    int64  
 5   trams              174 non-null    int64  
 6   fastEight          174 non-null    float64
 7   fastSixes          174 non-null    int64  
 8   fastQuads          174 non-null    int64  
 9   quad               174 non-null    int64  
 10  triple             174 non-null    int64  
 11  double             174 non-null    int64  
 12  surface            174 non-null    int64  
 13  total_chairs       174 non-null    int64  
 14  Runs               174 non-null    float64
 15  TerrainParks       174 non-null    float64
 16  LongestRun_mi      174 non

#### Model 3

In [114]:
df_new = df_new.drop(['summit_elev', 'vertical_drop'], axis=1)

In [115]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df_new.drop(['Name','AdultWeekend', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df_new['AdultWeekend'] 

In [116]:
# Let's standardize the magnitude by calling stdx() function we have written above
X_scaled = stdz(X)

In [117]:
# Let's split data by calling split_data() function we have written above
X_train, X_test, y_train, y_test = split_data(X_scaled, y)

In [118]:
model_3 = lm.fit(X_train,y_train)

In [119]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by=['Coefficient'], ascending=False).head(12)

,Coefficient
AdultWeekday,9.380163
clusters,2.372898
SkiableTerrain_ac,2.202828
averageSnowfall,2.090824
LongestRun_mi,2.076764
triple,1.503829
daysOpenLastYear,1.503031
surface,1.400994
Runs,1.109672
quad,1.095383


In [120]:
y_pred = model_3.predict(X_test)

In [121]:
evaluate(y_test, y_pred)

Explained Variance Score:  0.7661127719597833


Mean Absolute Error:  6.270091452299053


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | -2.70 | 22096951519.566845|-|
| Model 2. | 0.74 | 6.88 |'state'|
| Model 3. | 0.76 | 6.27 |'state','summit_elev','vertical_drop'|

Model Selection: Model 3 